In [2]:
import pandas as pd
import gzip

In [3]:
gtex_files = [['gtex_liver_mitocarta', 'gene_reads_v10_liver'],
              ['gtex_heart_aa_mitocarta', 'gene_reads_v10_heart_atrial_appendage'],
              ['gtex_heart_lv_mitocarta', 'gene_reads_v10_heart_left_ventricle']]

for gtex_file in gtex_files:
    # Ruta al archivo
    archivo = f'./data/04_htseq/human/before_mitocarta/{gtex_file[1]}.gct.gz'

    # Abrir el archivo comprimido
    with gzip.open(archivo, 'rt') as f:
        # Leer las primeras dos líneas (metadatos)
        f.readline()  # Versión
        f.readline()  # Dimensiones
        # Leer el resto como DataFrame
        gtex = pd.read_csv(f, sep='\t', index_col=0)

    # 1) Leer MitoCarta 3.0 (Excel humano)
    mc = pd.read_excel("./data/Human.MitoCarta3.0_clean.xls")  # ajusta ruta/nombre

    # Mira qué columna es el Ensembl ID (ej. 'Ensembl ID' o similar)
    # print(mc.columns)

    # 2) Lista de Ensembl de MitoCarta
    mito_ens = mc["EnsemblGeneID_mapping_version_20200130"].dropna().astype(str).str.strip().unique()

    # 3) Limpiar Ensembl IDs de GTEx (quitando versión)
    gtex = gtex.copy()
    gtex["ensembl_original"] = gtex.index  # se añadirá como última columna

    # Crear índice limpio solo para filtrar
    gtex.index = gtex.index.str.split(".").str[0]

    # 4) Filtrar solo genes MitoCarta
    gtex_mito = gtex.loc[gtex.index.intersection(mito_ens)]

    # Si quieres pasar de Ensembl a símbolo usando el Excel
    mc_sub = mc[["EnsemblGeneID_mapping_version_20200130", "Symbol"]]  # nombres de columnas según fichero
    mc_sub.columns = ["ensembl_clean", "symbol"]
    mc_sub = mc_sub.set_index("ensembl_clean")

    gtex_mito = gtex_mito.join(mc_sub, how="left")
    # aquí el índice sigue siendo Ensembl (limpio) y tienes una columna 'symbol'

    # 1) Añadir 'symbol' como columna (ya está) y reordenar columnas
    cols = gtex_mito.columns.tolist()

    # nos aseguramos de que 'Description' exista
    # y movemos 'symbol' justo después de 'Description'
    cols_reordered = (
        ["symbol", "ensembl_original", "Description"] +
        [c for c in cols if c not in ["symbol", "ensembl_original", "Description"]]
    )

    gtex_mito = gtex_mito[cols_reordered]

    # 2) Guardar a TSV manteniendo Ensembl como rownames
    gtex_mito.to_csv(f"./data/04_htseq/human/after_mitocarta/{gtex_file[0]}.tsv", sep="\t")